# Parse all PE to transform JSON files

In [ ]:
from peanalysis.parse import PEParser
import os
import json
import numpy as np
from IPython.display import display
from ipywidgets import widgets

## Loads all path of pe files

In [ ]:
input_text=widgets.Text()
all_file = []
path_repo = '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/'




In [ ]:
for root, subFolders, files in os.walk(path_repo):
    for file in files:
        path_file = os.path.join(root, file)
        all_file.append(path_file)

In [ ]:
all_file

##  Store all PEs parsed

In [ ]:
all_pe = []
for p in all_file:
    try:
        pe = PEParser(p)
        pe.load()
        pe.dump_sections()
        pe.dump_imports()
        pe.dump_exports()
        pe.dump_resources()
        pe.dump_tls()
        all_pe.append(pe)
    except:
        pass

In [ ]:
len(all_pe)

In [ ]:
all_pe[0].dict_pe

## Record in json files the data parsed

In [ ]:
for p in all_pe:
    json.dump( p.dict_pe,open('%s.json' %os.path.join('jsonfiles', p.dict_pe['hashes']['sha256']),'w'))

In [ ]:
len(all_pe)

## Check Collisions

In [ ]:
all_sha256 = [ p.dict_pe['hashes']['sha256']for p in all_pe]

In [ ]:
len(all_sha256)

In [ ]:
len(set(all_sha256))

In [ ]:
from collections import Counter
c = Counter()
for s in all_sha256:
    c[s] +=1

In [ ]:
collision = [ k for k,v in c.items() if v > 1]
collision

In [ ]:
[(p.dict_pe['hashes']['sha256'] ,p.dict_pe['path']) for p in all_pe if p.dict_pe['hashes']['sha256'] in collision]

In [ ]:
sorted(_, key = lambda x : x[0])

# First vector of features

## Storage metadata in redis

In [ ]:
import glob
from redis_management import RedisManagement
from collections import OrderedDict
from ml_helpers import ml_helpers
rmgt = RedisManagement('malwares')

In [ ]:
features = ['size of file', 'number of sections', 'median of entropy', 'number of imports', 'number of exports']

In [ ]:
all_json_files = [ json.load(open(path))for path in glob.glob('jsonfiles/*')]

In [ ]:
for j in all_json_files:
    malware = OrderedDict()
    malware[features[0]] = j['size']
    malware[features[1]] = j['number_sections']
    malware[features[2]] = sum( sec['entropy'] for sec in j['sections']) / j['number_sections']
    malware[features[3]] = sum(len(v) for k,v in j['imports'].items())
    malware[features[4]] =  sum(len(v) for k,v in j['exports'].items())
    rmgt.record_vector(j['hashes']['sha256'],'first_vector',malware)
    rmgt.record_malware_label(j['hashes']['sha256'],j['path'],path_repo)   

## Create Matrix of vectors features

In [ ]:
ml_h = ml_helpers(rmgt.redis_client)
mat = ml_h.create_matrix('first_vector')
mat

In [ ]:
mat.shape

In [ ]:
np.save('matrix_first_vector', mat)

# Second Vector of features

In [ ]:
import glob
from redis_management import RedisManagement
from collections import OrderedDict
from ml_helpers import ml_helpers
import numpy as np
rmgt = RedisManagement('malwares')
ml_h = ml_helpers(rmgt.redis_client)

In [ ]:
all_vectors = [eval(rmgt.client.hget(k,'first_vector')) for k in sorted(rmgt.client.keys())]

In [ ]:
all_vectors

In [ ]:
max_values = ml_h.get_max_values(all_vectors,features)

In [ ]:
max_values

In [ ]:
list_second_vector = [OrderedDict((k, v / max_values[k]) for k,v in m.items()) for m in all_vectors]

In [ ]:
list_second_vector

In [ ]:
all_malware=ml_h.get_all_malwares
for index,v in enumerate(list_second_vector):
    rmgt.record_vector(all_malware[index],'second_vector',v)
    

In [ ]:
mat_second_vect = ml_h.create_matrix('second_vector')

In [ ]:
mat_second_vect.shape

In [ ]:
np.save('matrix_second_vector',mat_second_vect)